# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis = 1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis = 1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis = 1)

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation = 'relu', input_shape = (n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation = 'relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation = 'linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

Epoch 137/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [===========

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns = numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns = numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns = numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [9]:
# Train the model
normalized_input_model.fit(X_train, y_train, batch_size = 32, epochs = 150, validation_data = (X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [====================

Epoch 137/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [===========

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, epochs = 150, batch_size = 32,
                    validation_data = (X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.5364 - mse: 0.5364 - val_loss: 0.2464 - val_mse: 0.2464
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2543 - mse: 0.2543 - val_loss: 0.1817 - val_mse: 0.1817
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2161 - mse: 0.2161 - val_loss: 0.1655 - val_mse: 0.1655
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1896 - mse: 0.1896 - val_loss: 0.1540 - val_mse: 0.1540
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1746 - mse: 0.1746 - val_loss: 0.1435 - val_mse: 0.1435
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1543 - mse: 0.1543 - val_loss: 0.1560 - val_mse: 0.1560
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1427 - mse: 0.1427 - val_loss: 0.1379 - val_mse: 0.1379
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1300 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.1587 - val_mse: 0.1587
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0245 - mse: 0.0245 - val_loss: 0.1585 - val_mse: 0.1585
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1598 - val_mse: 0.1598
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1574 - val_mse: 0.1574
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0232 - mse: 0.0232 - val_loss: 0.1588 - val_mse: 0.1588
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1550 - val_mse: 0.1550
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1577 - val_mse: 0.1577
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0225 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.1659 - val_mse: 0.1659
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1673 - val_mse: 0.1673
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1633 - val_mse: 0.1633
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1646 - val_mse: 0.1646
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1659 - val_mse: 0.1659
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.1648 - val_mse: 0.1648
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1643 - val_mse: 0.1643
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [12]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 720us/step - loss: 0.0087 - mse: 0.0087


[0.00873105600476265, 0.00873105600476265]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [13]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 806us/step - loss: 0.1670 - mse: 0.1670


[0.16701745986938477, 0.16701745986938477]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [14]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

32115.33201902953

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [15]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation= 'relu', input_shape = (n_features),
                   kernel_initializer = 'he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer = 'SGD', loss = 'mse', metrics = ['mse'])
# Train the model
he_model.fit(X_train, y_train_scaled, epochs= 150, batch_size = 32, 
             validation_data = (X_val, y_val_scaled)) 

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.5120 - mse: 0.5120 - val_loss: 0.2218 - val_mse: 0.2218
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2770 - mse: 0.2770 - val_loss: 0.1714 - val_mse: 0.1714
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2188 - mse: 0.2188 - val_loss: 0.1389 - val_mse: 0.1389
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1933 - mse: 0.1933 - val_loss: 0.1315 - val_mse: 0.1315
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1706 - mse: 0.1706 - val_loss: 0.1157 - val_mse: 0.1157
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1608 - mse: 0.1608 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1459 - mse: 0.1459 - val_loss: 0.1180 - val_mse: 0.1180
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1370 - m

Epoch 63/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.1280 - val_mse: 0.1280
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1273 - val_mse: 0.1273
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1267 - val_mse: 0.1267
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1277 - val_mse: 0.1277
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1277 - val_mse: 0.1277
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1331 - val_mse: 0.1331
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0214 - mse: 0.0214 - val_loss: 0.1275 - val_mse: 0.1275
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.

33/33 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1270 - val_mse: 0.1270
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1294 - val_mse: 0.1294
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1289 - val_mse: 0.1289
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1288 - val_mse: 0.1288
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1290 - val_mse: 0.1290
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1302 - val_mse: 0.1302
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0095 -

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 726us/step - loss: 0.0070 - mse: 0.0070


[0.00695146294310689, 0.00695146294310689]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [17]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 818us/step - loss: 0.1299 - mse: 0.1299


[0.12987393140792847, 0.12987393140792847]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [18]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu',
                             input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer = 'SGD', loss = 'mse', metrics = ['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4357 - mse: 0.4357 - val_loss: 0.1833 - val_mse: 0.1833
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2279 - mse: 0.2279 - val_loss: 0.1509 - val_mse: 0.1509
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1844 - mse: 0.1844 - val_loss: 0.1393 - val_mse: 0.1393
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1597 - mse: 0.1597 - val_loss: 0.1329 - val_mse: 0.1329
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1373 - mse: 0.1373 - val_loss: 0.1263 - val_mse: 0.1263
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1235 - mse: 0.1235 - val_loss: 0.1228 - val_mse: 0.1228
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1155 - mse: 0.1155 - val_loss: 0.1218 - val_mse: 0.1218
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1008 - m

Epoch 63/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0214 - mse: 0.0214 - val_loss: 0.1415 - val_mse: 0.1415
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0218 - mse: 0.0218 - val_loss: 0.1425 - val_mse: 0.1425
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1425 - val_mse: 0.1425
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0209 - mse: 0.0209 - val_loss: 0.1396 - val_mse: 0.1396
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0206 - mse: 0.0206 - val_loss: 0.1412 - val_mse: 0.1412
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0202 - mse: 0.0202 - val_loss: 0.1452 - val_mse: 0.1452
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0197 - mse: 0.0197 - val_loss: 0.1438 - val_mse: 0.1438
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.

33/33 [==============================] - 0s 2ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1456 - val_mse: 0.1456
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1441 - val_mse: 0.1441
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1447 - val_mse: 0.1447
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1459 - val_mse: 0.1459
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1455 - val_mse: 0.1455
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1424 - val_mse: 0.1424
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1446 - val_mse: 0.1446
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0096 -

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 673us/step - loss: 0.0072 - mse: 0.0072


[0.007164028938859701, 0.007164028938859701]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 804us/step - loss: 0.1460 - mse: 0.1460


[0.14601224660873413, 0.14601224660873413]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [22]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mse'])
# Code here

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.3396 - mse: 0.3396 - val_loss: 0.1074 - val_mse: 0.1074
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2008 - mse: 0.2008 - val_loss: 0.1093 - val_mse: 0.1093
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1585 - mse: 0.1585 - val_loss: 0.1026 - val_mse: 0.1026
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1249 - mse: 0.1249 - val_loss: 0.1008 - val_mse: 0.1008
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1076 - mse: 0.1076 - val_loss: 0.0925 - val_mse: 0.0925
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0870 - mse: 0.0870 - val_loss: 0.0919 - val_mse: 0.0919
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0726 - mse: 0.0726 - val_loss: 0.1521 - val_mse: 0.1521
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0672 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.1125 - val_mse: 0.1125
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1123 - val_mse: 0.1123
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0065 - mse: 0.0065 - val_loss: 0.1202 - val_mse: 0.1202
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0065 - mse: 0.0065 - val_loss: 0.1193 - val_mse: 0.1193
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1156 - val_mse: 0.1156
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.1199 - val_mse: 0.1199
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.1274 - val_mse: 0.1274
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0064 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.0968 - val_mse: 0.0968
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.0967 - val_mse: 0.0967
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.0999 - val_mse: 0.0999
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0840 - val_mse: 0.0840
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0051 - mse: 0.0051 - val_loss: 0.0942 - val_mse: 0.0942
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0919 - val_mse: 0.0919
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 731us/step - loss: 0.0045 - mse: 0.0045


[0.004474341869354248, 0.004474341869354248]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [24]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 699us/step - loss: 0.0842 - mse: 0.0842


[0.08417217433452606, 0.08417217433452606]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [25]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer = 'Adam', loss ='mse', metrics = ['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 8ms/step - loss: 0.4238 - mse: 0.4238 - val_loss: 0.1371 - val_mse: 0.1371
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1544 - mse: 0.1544 - val_loss: 0.1148 - val_mse: 0.1148
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1029 - mse: 0.1029 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0767 - mse: 0.0767 - val_loss: 0.1274 - val_mse: 0.1274
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0612 - mse: 0.0612 - val_loss: 0.1050 - val_mse: 0.1050
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0476 - mse: 0.0476 - val_loss: 0.0993 - val_mse: 0.0993
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0389 - mse: 0.0389 - val_loss: 0.1025 - val_mse: 0.1025
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0311 - m

Epoch 63/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0029 - mse: 0.0029 - val_loss: 0.0924 - val_mse: 0.0924
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.0934 - val_mse: 0.0934
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.0835 - val_mse: 0.0835
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0085 - mse: 0.0085 - val_loss: 0.0961 - val_mse: 0.0961
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.0901 - val_mse: 0.0901
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0885 - val_mse: 0.0885
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0979 - val_mse: 0.0979
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.

33/33 [==============================] - 0s 2ms/step - loss: 0.0056 - mse: 0.0056 - val_loss: 0.0856 - val_mse: 0.0856
Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.0858 - val_mse: 0.0858
Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0034 - mse: 0.0034 - val_loss: 0.0841 - val_mse: 0.0841
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.0829 - val_mse: 0.0829
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.0838 - val_mse: 0.0838
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 7.1235e-04 - mse: 7.1235e-04 - val_loss: 0.0813 - val_mse: 0.0813
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 4.6046e-04 - mse: 4.6046e-04 - val_loss: 0.0818 - val_mse: 0.0818
Epoch 130/150
33/33 [==============================] - 0s 2ms/step 

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 681us/step - loss: 0.0027 - mse: 0.0027


[0.002652422059327364, 0.002652422059327364]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [27]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 669us/step - loss: 0.0826 - mse: 0.0826


[0.08263609558343887, 0.08263609558343887]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [28]:
# Evaluate the best model on test data
adam_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 828us/step - loss: 0.1687 - mse: 0.1687


[0.16872599720954895, 0.16872599720954895]

In [29]:
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 804us/step - loss: 0.1452 - mse: 0.1452


[0.14517812430858612, 0.14517812430858612]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [30]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

29942.086219456243

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.